# Simple Evolutionary Exploration  Walkthrough

This notebook contains instructions on how to use the SEE module, along with several examples. These instructions will cover the following parts: 
* [Import Image Files](#Import_Image_Files)
* [Manual Search](#Manual_Search)
* [Genetic Algorithm Search](#Genetic_Algorithm_Search)
* [Reading the Results](#Reading_the_Results)

These examples use the Jupyter widgets to make it interactive and easier to use and learn.

----
<a name="Import_Image_Files"></a>

## Import Image Files

First import the following packages:

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt
from imageio import v3 as imageio
from see import Segmentors
from see import JupyterGUI

Pick an image from the example folder.

In [ ]:
data = JupyterGUI.pickimage('Image_data/Examples/')

# Select ColorSpace

In [ ]:
colorspace = JupyterGUI.colorwidget(data.img, paramlist=None)

----
<a name="Manual_Search"></a>

## Manual Search

Manual searching of parameters can easily be done using the provided GUI. Pre-established parameters can be put into the widget, or the parameter values can be changed using the sliders. To change the algorithm, simply change the `alg` input. For a list of available inputs print `Segmentors.algorithmspace`


In [ ]:
from see.Segmentors import segmentor

alg = JupyterGUI.picksegment(list(segmentor.algorithmspace.keys()))

In [ ]:
### Example of input for params
params = JupyterGUI.segmentwidget(data.img, params = None, alg = alg.value)

----
<a name="Genetic_Algorithm_Search"></a>

## Genetic Algorithm Search

First import image files, as well as the following packages:

In [ ]:
from see.Segmentors import segmentor
from see.ColorSpace import colorspace
from see.Workflow import workflow
from see.Segment_Fitness import segment_fitness
from see import base_classes, GeneticSearch
#define algo work flow
workflow.addalgos([colorspace, segmentor, segment_fitness])

To run the genetic algorithm, we need to initialize an instance of an evolver. The original image and ground truth segmentation image are inputs to it, along with an integer value for population size. This value sets how many indivudals are in our population. For this example, we'll set this number to be equal to 10.

In [ ]:
mydata = base_classes.pipedata()
mydata.append(data.img)
mydata.gtruth = data.gmask
#def geneticsearch
my_evolver = GeneticSearch.Evolver(workflow, mydata, pop_size=10)

Now that the evolver has been initialized, we can run the genetic algorithm for a specified number of generations (or iterations). Here we will set this number equal to 5.

In [ ]:
# warnings may appear when this runs
#population -- Resulting population after ngen generations.
population = my_evolver.run(ngen=1) #5 generation
#fitness 1 is worse , 0 is good

<class 'see.Workflow.workflow'> parameters: 
	colorspace = YUV
	multichannel = False
	channel = 0
	algorithm = Felzenszwalb
	alpha1 = 0.1875
	alpha2 = 0.921875
	beta1 = 0.74609375
	beta2 = 0.296875
	gamma1 = 0.609375
	gamma2 = 0.46875
	n_segments = 8
	max_iter = 10

Time: 4.706 s
fitness=0.13652921828966982


#BEST [0.13652921828966982,  ['YUV', False, 0, 'Felzenszwalb', 0.1875, 0.921875, 0.74609375, 0.296875, 0.609375, 0.46875, 8, 10]]

----
<a name="Reading_the_Results"></a>

## Reading the Results

After the genetic algorithm is complete, we can retrieve the individuals that resulted in the lowest (best) fitness values by printing `my_evolver.hof`. These individuals are sorted according to fitness value, so to get the overal best individual, we can simply look at the first individual in the list. 

In [ ]:
params = my_evolver.hof[0]

print('Best Individual:\n', params)

We can see the mask this individual generates by evaluating it, then plotting the result:

In [ ]:
#['YUV', False, 0, 'Felzenszwalb', 0.1875, 0.921875, 0.74609375, 0.296875, 0.609375, 0.46875, 8, 10]
#------------------------------------
#this line is the original code
#seg = Segmentors.algoFromParams(params)
# no attribute  .algoFromParams
#------------------------------------
#what if the class is called wrong: 
#seg = segmentor(param)
# KeyError: 'YUV'
#------------------------------------

#use workflow instead, to create a workf class
#since the parameter input might produce error, use the workflow's parameter to make a seg object
workf = workflow(params)

mydata = workf.pipe(mydata)

plt.figure(figsize=(20, 10))
plt.subplot(121)
plt.imshow(mydata[0])
plt.title("Original Image")
plt.axis('off')

plt.subplot(122)
plt.imshow(mydata[-1])
plt.title("Segmentation")
plt.axis('off')

plt.tight_layout
plt.show()

In [ ]:
print(f"Fitnessvalue = {mydata.fitness}")

With this code, make sure to import skimage, along with any input images this algorithm will be applied to.